In [1]:
import os
import sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, os.path.join(cwd, "../"))

import pickle
import torch
import importlib
import numpy as np
import pandas as pd
import copy

import src.bin.tensorify as tensorify
import src.utils.data_conversion_utils as conversions
import src.data_manager.student_life_var_binned_data_manager as data_manager
import src.bin.trainer as trainer

from sklearn import metrics

from torch import nn
from copy import copy
from copy import deepcopy
from src import definitions
from src.bin import validations
from src.bin import statistics
from src.bin import plotting
from src.utils.read_utils import read_pickle
from src.utils import student_utils
from src.data_manager import sub_sampler
from src.data_manager import cross_val
from src.data_manager import helper as data_manager_helper
from tabulate import tabulate
from src.models import autoencoder
from src.models import multitask_learning
from src.models import user_dense_heads

importlib.reload(validations)
importlib.reload(autoencoder)
importlib.reload(multitask_learning)
importlib.reload(user_dense_heads)
importlib.reload(sub_sampler)
importlib.reload(statistics)
importlib.reload(cross_val)
importlib.reload(definitions)
importlib.reload(conversions)
importlib.reload(tensorify)
importlib.reload(plotting)
importlib.reload(trainer)
importlib.reload(data_manager)
importlib.reload(student_utils)
importlib.reload(data_manager_helper)

pd.set_option('max_rows', 10000)
feature_list = data_manager.FEATURE_LIST

In [6]:
# ##### Pickle #####
# data = read_pickle('../data/training_data/most_representative_6_hr_b_imputed_normalized_prev_stress_data.pkl')

#### Data ####
# student_list = [53, 46, 7, 49, 22, 24, 2]
# student_list = [ 7]
# data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=True, 
#                                                          fill_na=True, flatten_sequence=False)

# ###### Randomize ######
# train_ids, val_ids, test_ids = cross_val.random_stratified_splits(data)
# data['train_ids'] = train_ids
# data['val_ids'] = val_ids
# data['test_ids'] = test_ids

# data = sub_sampler.get_sub_sampled_sequences(data)

############# Stats ############# 
# unnormalized_data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=False, fill_na=False)
# statistics_df, raw_df = statistics.get_statistics_on_data_dict(unnormalized_data, feature_list)
# print(statistics.get_train_test_val_label_counts_from_raw_data(data))


#### Data ####
student_list = [53, 46, 7, 49, 22, 24, 2]
student_list = [2,  46, 7]
# student_list = [ 7]
data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=False, 
                                                         fill_na=True, flatten_sequence=False)

###### Randomize ######
train_ids, val_ids, test_ids = cross_val.random_stratified_splits(data, stratify_by='students')
data['train_ids'] = train_ids
data['val_ids'] = val_ids
data['test_ids'] = test_ids

# data = sub_sampler.get_sub_sampled_sequences(data)

############ Stats ############# 
# unnormalized_data = data_manager.get_data_for_training_in_dict_format(*student_list, normalize=False, fill_na=False)
# statistics_df, raw_df = statistics.get_statistics_on_data_dict(unnormalized_data, feature_list)
print(statistics.get_train_test_val_label_counts_from_raw_data(data))

Student: 2
Before Midterm
Train Set: ['3_28_22', '3_29_23', '3_30_22', '3_31_22', '4_4_1', '4_5_6', '4_8_2', '4_8_20', '4_9_8']
Val Set: ['4_14_22', '4_15_20']
Test Set: ['4_16_4', '4_16_9']
During Midterm
Train Set: ['4_17_8', '4_18_4']
Val Set: []
Test Set: ['4_30_4']
AFter Midterm
Train Set: []
Val Set: []
Test Set: []
Student: 46
Before Midterm
Train Set: ['4_2_9', '4_3_0', '4_3_6', '4_4_0', '4_4_6', '4_5_0', '4_5_6', '4_6_0', '4_6_7', '4_7_2', '4_8_3', '4_8_20', '4_9_8']
Val Set: ['4_12_22', '4_13_22']
Test Set: []
During Midterm
Train Set: []
Val Set: []
Test Set: ['4_30_4']
AFter Midterm
Train Set: []
Val Set: []
Test Set: []
Student: 7
Before Midterm
Train Set: ['3_31_2', '3_31_7', '3_31_9', '3_31_12', '3_31_21', '3_31_22', '4_1_4', '4_1_6', '4_4_1', '4_4_13', '4_5_8']
Val Set: ['4_12_2', '4_12_22', '4_13_22']
Test Set: ['4_16_4']
During Midterm
Train Set: ['4_17_3', '4_17_8', '4_18_0', '4_18_10']
Val Set: []
Test Set: []
AFter Midterm
Train Set: []
Val Set: []
Test Set: []
   

In [ ]:
data_df = pd.DataFrame(data['data']['24_4_9_19'][definitions.HISTOGRAM_IDX])
print(type(data_df.mean()))

In [ ]:
sequence_len, num_features, histogram_seq_len, hist_num_feats = statistics.get_sequence_length_and_num_features_from_data(data)
print("sequence_len: {} num_features: {} histogram_seq_len: {} histogram_num_feats: {}".format(
        sequence_len, num_features, histogram_seq_len, hist_num_feats))


In [7]:
display(pd.DataFrame(data['data']['7_3_31_7'][definitions.HISTOGRAM_IDX]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,3.533333,2.883333,40.866667,52.0,0.0,0.0,0.0,52.0,0.0,0.0,0.0,52.0,0.0,52.0,0.0,0.0,52.0,0.0,52.0
1,1.0,4.400000,2.016667,40.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
2,1.0,5.400000,1.016667,39.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
3,1.0,6.400000,0.016667,38.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
4,1.0,0.983333,15.116667,37.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
5,2.0,1.983333,14.116667,36.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
6,2.0,2.983333,13.116667,35.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
7,2.0,3.983333,12.116667,34.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
8,2.0,4.983333,11.116667,33.000000,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0
9,2.0,5.983333,10.116667,32.000000,58.0,2.0,0.0,0.0,51.0,0.0,9.0,0.0,60.0,0.0,60.0,0.0,0.0,60.0,0.0,60.0


In [ ]:
print(data['data'].keys())